In [127]:
import pandas as pd
import altair as alt
from sklearn.model_selection import train_test_split

In [9]:
# Reading data from 1997 to 2012

old_data = pd.read_csv("Data/business-licences-1997-to-2012.csv", sep = ";")

In [77]:
old_data.FOLDERYEAR.unique()

array([ 6.,  5.,  7.,  4.,  9.,  8., 10., 11.,  3.,  2.,  0.,  1., 99.,
       12., 97., 96., 98., 94., nan])

In [10]:
# Reading new data from 2013 to 2020

new_data = pd.read_csv("Data/business-licences.csv", sep = ";")

In [145]:
# Training set contains data from 1997 up until 2017

train = pd.concat([old_data, new_data[new_data.FOLDERYEAR.isin([13, 14, 15])]])

In [131]:
validation = pd.concat([old_data, new_data[new_data.FOLDERYEAR.isin([16, 17])]])

In [42]:
# Test set contains data for 2019 and 2020

test = new_data[new_data.FOLDERYEAR.isin([18, 19, 20])]

## EDA

In [147]:
train.dtypes

FOLDERYEAR               float64
LicenceRSN                 int64
LicenceNumber             object
LicenceRevisionNumber    float64
BusinessName              object
BusinessTradeName         object
Status                    object
IssuedDate                object
ExpiredDate               object
BusinessType              object
BusinessSubType           object
Unit                      object
UnitType                  object
House                     object
Street                    object
City                      object
Province                  object
Country                   object
PostalCode                object
LocalArea                 object
NumberofEmployees        float64
FeePaid                  float64
ExtractDate               object
Geom                      object
dtype: object

In [165]:
train.shape[0] - train.LicenceRSN.nunique()

772

> Some duplicate RSNs.

In [149]:
train.groupby(by = "FOLDERYEAR").LicenceNumber.nunique()

FOLDERYEAR
0.0     58278
1.0     60258
2.0     58636
3.0     59432
4.0     59048
5.0     59674
6.0     58464
7.0     57578
8.0     58134
9.0     59096
10.0    60193
11.0    60494
12.0    59770
13.0    59248
14.0    59076
15.0    59447
94.0        1
96.0       10
97.0    63239
98.0    63423
99.0    59581
Name: LicenceNumber, dtype: int64

> I believe that years 94 and 96 are not useful and can be dropped

In [150]:
train = train.query("FOLDERYEAR != [94.0, 96.0]")

In [163]:
train.isna().any()

FOLDERYEAR                True
LicenceRSN               False
LicenceNumber            False
LicenceRevisionNumber     True
BusinessName              True
BusinessTradeName         True
Status                   False
IssuedDate                True
ExpiredDate               True
BusinessType             False
BusinessSubType           True
Unit                      True
UnitType                  True
House                     True
Street                    True
City                      True
Province                  True
Country                   True
PostalCode                True
LocalArea                 True
NumberofEmployees        False
FeePaid                   True
ExtractDate              False
Geom                      True
dtype: bool

> Many missing values. Luckily, non in LicenceRSN.

> I should only consider Businesses in Vancouver by filtering according to city.

> Another note from the data source to keep in mind.
> * A small percentage of businesses are issued with a replacement licence in a displayed year due to changes or when business type names were retired during mid-year.